# Problem Statement
## **Business Context**
"Visit with Us," a leading travel company, is revolutionizing the tourism industry by leveraging data-driven strategies to optimize operations and customer engagement. While introducing a new package offering, such as the Wellness Tourism Package, the company faces challenges in targeting the right customers efficiently. The manual approach to identifying potential customers is inconsistent, time-consuming, and prone to errors, leading to missed opportunities and suboptimal campaign performance.

To address these issues, the company aims to implement a scalable and automated system that integrates customer data, predicts potential buyers, and enhances decision-making for marketing strategies. By utilizing an MLOps pipeline, the company seeks to achieve seamless integration of data preprocessing, model development, deployment, and CI/CD practices for continuous improvement. This system will ensure efficient targeting of customers, timely updates to the predictive model, and adaptation to evolving customer behaviors, ultimately driving growth and customer satisfaction.

## **Objective**
As an MLOps Engineer at "Visit with Us," your responsibility is to design and deploy an MLOps pipeline on GitHub to automate the end-to-end workflow for predicting customer purchases. The primary objective is to build a model that predicts whether a customer will purchase the newly introduced Wellness Tourism Package before contacting them. The pipeline will include data cleaning, preprocessing, transformation, model building, training, evaluation, and deployment, ensuring consistent performance and scalability. By leveraging GitHub Actions for CI/CD integration, the system will enable automated updates, streamline model deployment, and improve operational efficiency. This robust predictive solution will empower policymakers to make data-driven decisions, enhance marketing strategies, and effectively target potential customers, thereby driving customer acquisition and business growth.

## **Data Description**
The dataset contains customer and interaction data that serve as key attributes for predicting the likelihood of purchasing the Wellness Tourism Package. The detailed attributes are:



**Customer Details**

- **CustomerID:** Unique identifier for each customer.

- **ProdTaken:** Target variable indicating whether the customer has purchased a package (0: No, 1: Yes).

- **Age:** Age of the customer.

- **TypeofContact:** The method by which the customer was contacted (Company Invited or Self Inquiry).

- **CityTier:** The city category based on development, population, and living standards (Tier 1 > Tier 2 > Tier 3).

- **Occupation:** Customer's occupation (e.g., Salaried, Freelancer).

- **Gender:** Gender of the customer (Male, Female).

- **NumberOfPersonVisiting:** Total number of people accompanying the customer on the trip.

- **PreferredPropertyStar:** Preferred hotel rating by the customer.

- **MaritalStatus:** Marital status of the customer (Single, Married, Divorced).

- **NumberOfTrips:** Average number of trips the customer takes annually.

- **Passport:** Whether the customer holds a valid passport (0: No, 1: Yes).

- **OwnCar:** Whether the customer owns a car (0: No, 1: Yes).

- **NumberOfChildrenVisiting:** Number of children below age 5 accompanying the customer.

- **Designation:** Customer's designation in their current organization.

- **MonthlyIncome:** Gross monthly income of the customer.



**Customer Interaction Data**

- **PitchSatisfactionScore:** Score indicating the customer's satisfaction with the sales pitch.

- **ProductPitched:** The type of product pitched to the customer.

- **NumberOfFollowups:** Total number of follow-ups by the salesperson after the sales pitch.-

- **DurationOfPitch:** Duration of the sales pitch delivered to the customer.

# Prerequisites
* Create a GitHub repo

    - Go to ***GitHub Profile***

    - Click on ***Your repositories*** then select ***New***

      - Repository Name: ***MLOps***

      - Check the box ***README.md*** file

      - Click on ***Create repository***



* Adding Hugging Face space secrets to GitHub Actions to execute the workflow

  1. Go to Hugging Face ***Profile***

  2. Navigate to ***Access Token***

  3. Create a ***New token***

      - Token type ***Write***

      - Token Name ***MLOps***

      - Click on ***Create Token***

      - Copy the generated Token

  4. Now, go to GitHub repo

      - Click on ***Settings***

      - Navigate to ***Secrets and Variables***

      - Click on ***Actions***

      - Add a ***Repository secerts***

        - Name ***HF_TOKEN***

        - Secret: ***Paste the token created from the hugging face access tokens***

        - Click on ***Add secret***



* Create a Hugging Face space

    - Go to **Hugging Face**

    - Open your **Profile**

    - Click on **New Space**

      - Under the space creation, enter the below details

        - Space name: **tourism-package-prediction**

    (If you were trying with different names, be cautious when using a underscore `_` in space names, such as `frontend_space`, as it can cause exceptions when accessing the API URL. Always use an hyphen `-` instead, like `frontend-space`.)

        - Select the space SDK: **Docker**

        - Choose a Docker template: **Streamlit**

        - Click on **Create Space**

# Create a master folder to keep all files created when executing the below code cells

In [ ]:
import os

os.makedirs("tourism_project", exist_ok=True)

# Model Building
## Data Registration

In [ ]:
os.makedirs("tourism_project/data", exist_ok=True)

Once the **data** folder created after executing the above cell, please upload the **tourism.csv** in to the folder

# Create a folder for storing the model building files

In [ ]:
os.makedirs("tourism_project/model_building", exist_ok=True)

In [ ]:
%%writefile tourism_project/model_building/data_register.py

from huggingface_hub.utils import RepositoryNotFoundError, HfHubHTTPError
from huggingface_hub import HfApi, create_repo
import os



repo_id = "<-----Hugging Face User ID ----->/tourism-package-prediction-data"
repo_type = "dataset"

# Initialize API client
api = HfApi(token=os.getenv("HF_TOKEN"))

# Step 1: Check if the space exists
try:
    api.repo_info(repo_id=repo_id, repo_type=repo_type)
    print(f"Space '{repo_id}' already exists. Using it.")
except RepositoryNotFoundError:
    print(f"Space '{repo_id}' not found. Creating new space...")
    create_repo(repo_id=repo_id, repo_type=repo_type, private=False)
    print(f"Space '{repo_id}' created.")

api.upload_folder(
    folder_path="tourism_project/data",
    repo_id=repo_id,
    repo_type=repo_type,
)

## Data Preparation

In [ ]:
%%writefile tourism_project/model_building/prep.py

# for data manipulation
import pandas as pd
import numpy as np
# for creating a folder
import os
# for data preprocessing and pipeline creation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# for hugging face space authentication to upload files
from huggingface_hub import login, HfApi

# Define constants for the dataset and output paths
api = HfApi(token=os.getenv("HF_TOKEN"))
DATASET_PATH = "hf://datasets/<-----Hugging Face User ID ----->/tourism-package-prediction-data/tourism.csv"
tourism_dataset = pd.read_csv(DATASET_PATH)
print("Dataset loaded successfully.")

# Remove unnecessary columns
if 'Unnamed: 0' in tourism_dataset.columns:
    tourism_dataset = tourism_dataset.drop('Unnamed: 0', axis=1)
if 'CustomerID' in tourism_dataset.columns:
    tourism_dataset = tourism_dataset.drop('CustomerID', axis=1)

# Handle missing values
# Fill numerical missing values with median
numerical_cols = tourism_dataset.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if tourism_dataset[col].isnull().sum() > 0:
        tourism_dataset[col].fillna(tourism_dataset[col].median(), inplace=True)

# Fill categorical missing values with mode
categorical_cols = tourism_dataset.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if tourism_dataset[col].isnull().sum() > 0:
        tourism_dataset[col].fillna(tourism_dataset[col].mode()[0], inplace=True)

# Encode categorical features using LabelEncoder
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    tourism_dataset[col] = le.fit_transform(tourism_dataset[col].astype(str))
    label_encoders[col] = le

# Define the target variable for the classification task
target = 'ProdTaken'

# Define predictor matrix (X) and target variable (y)
X = tourism_dataset.drop(target, axis=1)
y = tourism_dataset[target]

# Split dataset into train and test
# Split the dataset into training and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,              # Predictors (X) and target variable (y)
    test_size=0.2,     # 20% of the data is reserved for testing
    random_state=42,   # Ensures reproducibility by setting a fixed random seed
    stratify=y         # Stratify to maintain class distribution
)

Xtrain.to_csv("Xtrain.csv", index=False)
Xtest.to_csv("Xtest.csv", index=False)
ytrain.to_csv("ytrain.csv", index=False)
ytest.to_csv("ytest.csv", index=False)

files = ["Xtrain.csv", "Xtest.csv", "ytrain.csv", "ytest.csv"]

for file_path in files:
    api.upload_file(
        path_or_fileobj=file_path,
        path_in_repo=file_path.split("/")[-1],  # just the filename
        repo_id="<-----Hugging Face User ID ----->/tourism-package-prediction-data",
        repo_type="dataset",
    )

## Model Training
### Experimentation and Tracking (Development Environment)

In [ ]:
!pip install mlflow==3.0.1 pyngrok==7.2.12 -q

To get the ngrok authorization token, please go to this [link](https://dashboard.ngrok.com/authtokens), generate a new token, copy it, and paste it in the designated code line below.

In [ ]:
from pyngrok import ngrok
import subprocess
import mlflow

# Set your auth token here (replace with your actual token)
ngrok.set_auth_token("<-----Ngrok Authentication Token ----->")

# Start MLflow UI on port 5000
process = subprocess.Popen(["mlflow", "ui", "--port", "5000"])

# Create public tunnel
public_url = ngrok.connect(5000).public_url
print("MLflow UI is available at:", public_url)

In [ ]:
# Set the tracking URL for MLflow
mlflow.set_tracking_uri(public_url)

# Set the name for the experiment
mlflow.set_experiment("Tourism_Package_Prediction_Experiment")

In [ ]:
import pandas as pd
import numpy as np
# for creating a folder
import os
# for data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# for model training, tuning, and evaluation
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# for model serialization
import joblib

tourism_dataset = pd.read_csv("tourism_project/data/tourism.csv")
print("Dataset loaded successfully.")

# Remove unnecessary columns
if 'Unnamed: 0' in tourism_dataset.columns:
    tourism_dataset = tourism_dataset.drop('Unnamed: 0', axis=1)
if 'CustomerID' in tourism_dataset.columns:
    tourism_dataset = tourism_dataset.drop('CustomerID', axis=1)

# Handle missing values
numerical_cols = tourism_dataset.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if tourism_dataset[col].isnull().sum() > 0:
        tourism_dataset[col].fillna(tourism_dataset[col].median(), inplace=True)

categorical_cols = tourism_dataset.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if tourism_dataset[col].isnull().sum() > 0:
        tourism_dataset[col].fillna(tourism_dataset[col].mode()[0], inplace=True)

# Encode categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    tourism_dataset[col] = le.fit_transform(tourism_dataset[col].astype(str))
    label_encoders[col] = le

# Define the target variable for the classification task
target = 'ProdTaken'

# Define predictor matrix (X) and target variable (y)
X = tourism_dataset.drop(target, axis=1)
y = tourism_dataset[target]

# Split dataset into train and test
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Train multiple models with MLflow tracking
models = {
    'Decision_Tree': DecisionTreeClassifier(max_depth=10, min_samples_split=20, min_samples_leaf=10, random_state=42),
    'Random_Forest': RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_split=10, min_samples_leaf=5, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    'Gradient_Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, min_child_weight=3, subsample=0.8, colsample_bytree=0.8, random_state=42, use_label_encoder=False, eval_metric='logloss')
}

results = {}

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Train model
        model.fit(Xtrain, ytrain)
        
        # Make predictions
        y_pred = model.predict(Xtest)
        y_pred_proba = model.predict_proba(Xtest)[:, 1]
        
        # Calculate metrics
        metrics = {
            'accuracy': accuracy_score(ytest, y_pred),
            'precision': precision_score(ytest, y_pred, average='weighted'),
            'recall': recall_score(ytest, y_pred, average='weighted'),
            'f1_score': f1_score(ytest, y_pred, average='weighted'),
            'roc_auc': roc_auc_score(ytest, y_pred_proba)
        }
        
        # Log parameters and metrics
        mlflow.log_params(model.get_params())
        mlflow.log_metrics(metrics)
        
        # Log model
        mlflow.sklearn.log_model(model, "model")
        
        results[model_name] = {'model': model, 'metrics': metrics}
        print(f"{model_name} - Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}")

- As we can see, all the experiments conducted during model training are being logged by MLflow.

- Upon clicking the links in the output of the above code cell, we can check the tracking on MLflow.

Now that we've tested the experimentation tracking with MLflow in a development environment, let's convert this to the required script for production environment usage.

### Experimentation and Tracking (Production Environment)

In [ ]:
%%writefile tourism_project/model_building/train.py

# for data manipulation
import pandas as pd
import numpy as np
# for model training, tuning, and evaluation
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# for model serialization
import joblib
# for creating a folder
import os
# for hugging face space authentication to upload files
from huggingface_hub import login, HfApi, create_repo
from huggingface_hub.utils import RepositoryNotFoundError, HfHubHTTPError
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("tourism-package-prediction-experiment")

api = HfApi()

Xtrain_path = "hf://datasets/<-----Hugging Face User ID ----->/tourism-package-prediction-data/Xtrain.csv"
Xtest_path = "hf://datasets/<-----Hugging Face User ID ----->/tourism-package-prediction-data/Xtest.csv"
ytrain_path = "hf://datasets/<-----Hugging Face User ID ----->/tourism-package-prediction-data/ytrain.csv"
ytest_path = "hf://datasets/<-----Hugging Face User ID ----->/tourism-package-prediction-data/ytest.csv"

Xtrain = pd.read_csv(Xtrain_path)
Xtest = pd.read_csv(Xtest_path)
ytrain = pd.read_csv(ytrain_path)
ytest = pd.read_csv(ytest_path)

# Train multiple models with MLflow tracking
models = {
    'Decision_Tree': DecisionTreeClassifier(max_depth=10, min_samples_split=20, min_samples_leaf=10, random_state=42),
    'Random_Forest': RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_split=10, min_samples_leaf=5, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    'Gradient_Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, min_child_weight=3, subsample=0.8, colsample_bytree=0.8, random_state=42, use_label_encoder=False, eval_metric='logloss')
}

results = {}
best_f1_score = 0
best_model_name = None
best_model = None

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Train model
        model.fit(Xtrain, ytrain.values.ravel())
        
        # Make predictions
        y_pred = model.predict(Xtest)
        y_pred_proba = model.predict_proba(Xtest)[:, 1]
        
        # Calculate metrics
        metrics = {
            'accuracy': accuracy_score(ytest, y_pred),
            'precision': precision_score(ytest, y_pred, average='weighted'),
            'recall': recall_score(ytest, y_pred, average='weighted'),
            'f1_score': f1_score(ytest, y_pred, average='weighted'),
            'roc_auc': roc_auc_score(ytest, y_pred_proba)
        }
        
        # Log parameters and metrics
        mlflow.log_params(model.get_params())
        mlflow.log_metrics(metrics)
        
        # Log model
        mlflow.sklearn.log_model(model, "model")
        
        results[model_name] = {'model': model, 'metrics': metrics}
        print(f"{model_name} - Accuracy: {metrics['accuracy']:.4f}, F1-Score: {metrics['f1_score']:.4f}")
        
        # Track best model based on F1-score
        if metrics['f1_score'] > best_f1_score:
            best_f1_score = metrics['f1_score']
            best_model_name = model_name
            best_model = model

print(f"\nBest Model: {best_model_name} with F1-Score: {best_f1_score:.4f}")

# Save the best model locally
model_path = "best_tourism_model_v1.joblib"
joblib.dump(best_model, model_path)

# Log the model artifact
print(f"Model saved as artifact at: {model_path}")

# Upload to Hugging Face
repo_id = "<-----Hugging Face User ID ----->/tourism-package-prediction-model"
repo_type = "model"

# Step 1: Check if the space exists
try:
    api.repo_info(repo_id=repo_id, repo_type=repo_type)
    print(f"Space '{repo_id}' already exists. Using it.")
except RepositoryNotFoundError:
    print(f"Space '{repo_id}' not found. Creating new space...")
    create_repo(repo_id=repo_id, repo_type=repo_type, private=False)
    print(f"Space '{repo_id}' created.")

api.upload_file(
    path_or_fileobj="best_tourism_model_v1.joblib",
    path_in_repo="best_tourism_model_v1.joblib",
    repo_id=repo_id,
    repo_type=repo_type,
)

# Deployment
## Dockerfile

In [ ]:
os.makedirs("tourism_project/deployment", exist_ok=True)

In [ ]:
%%writefile tourism_project/deployment/Dockerfile

# Use a minimal base image with Python 3.9 installed
FROM python:3.9

# Set the working directory inside the container to /app
WORKDIR /app

# Copy all files from the current directory on the host to the container's /app directory
COPY . .

# Install Python dependencies listed in requirements.txt
RUN pip3 install -r requirements.txt

RUN useradd -m -u 1000 user
USER user
ENV HOME=/home/user \
	PATH=/home/user/.local/bin:$PATH

WORKDIR $HOME/app

COPY --chown=user . $HOME/app

# Define the command to run the Streamlit app on port "8501" and make it accessible externally
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0", "--server.enableXsrfProtection=false"]

## Streamlit App

In [ ]:
%%writefile tourism_project/deployment/app.py

import streamlit as st
import pandas as pd
from huggingface_hub import hf_hub_download
import joblib

# Download the model from the Model Hub
model_path = hf_hub_download(repo_id="<-----Hugging Face User ID ----->/tourism-package-prediction-model", filename="best_tourism_model_v1.joblib")

# Load the model
model = joblib.load(model_path)

# Streamlit UI for Tourism Package Prediction
st.title("Tourism Package Purchase Prediction App")
st.write("The Tourism Package Purchase Prediction App is an internal tool for travel company staff that predicts whether customers are likely to purchase the Wellness Tourism Package based on their details.")
st.write("Kindly enter the customer details to check whether they are likely to purchase the package.")

# Collect user input
Age = st.number_input("Age (customer's age in years)", min_value=18, max_value=100, value=35)
TypeofContact = st.selectbox("Type of Contact (how the customer was contacted)", ["Company Invited", "Self Enquiry"])
CityTier = st.selectbox("City Tier (city category based on development)", [1, 2, 3])
DurationOfPitch = st.number_input("Duration of Pitch (in minutes)", min_value=0.0, max_value=60.0, value=15.0)
Occupation = st.selectbox("Occupation (customer's occupation)", ["Salaried", "Small Business", "Large Business", "Free Lancer"])
Gender = st.selectbox("Gender (customer's gender)", ["Male", "Female"])
NumberOfPersonVisiting = st.number_input("Number of Persons Visiting", min_value=1, max_value=10, value=2)
NumberOfFollowups = st.number_input("Number of Follow-ups", min_value=0.0, max_value=10.0, value=3.0)
ProductPitched = st.selectbox("Product Pitched (type of product pitched)", ["Basic", "Standard", "Deluxe", "Super Deluxe", "King"])
PreferredPropertyStar = st.selectbox("Preferred Property Star (preferred hotel rating)", [3.0, 4.0, 5.0])
MaritalStatus = st.selectbox("Marital Status (customer's marital status)", ["Single", "Married", "Divorced", "Unmarried"])
NumberOfTrips = st.number_input("Number of Trips (average trips per year)", min_value=0.0, max_value=20.0, value=2.0)
Passport = st.selectbox("Has Passport?", ["Yes", "No"])
PitchSatisfactionScore = st.selectbox("Pitch Satisfaction Score", [1, 2, 3, 4, 5])
OwnCar = st.selectbox("Owns Car?", ["Yes", "No"])
NumberOfChildrenVisiting = st.number_input("Number of Children Visiting (below age 5)", min_value=0, max_value=5, value=0)
Designation = st.selectbox("Designation (customer's designation)", ["Executive", "Manager", "Senior Manager", "AVP", "VP"])
MonthlyIncome = st.number_input("Monthly Income (gross monthly income)", min_value=0.0, value=20000.0)

# Encoding mappings (based on training data)
type_of_contact_map = {'Company Invited': 0, 'Self Enquiry': 1}
occupation_map = {'Salaried': 0, 'Small Business': 1, 'Large Business': 2, 'Free Lancer': 3}
gender_map = {'Male': 0, 'Female': 1}
product_pitched_map = {'Basic': 0, 'Standard': 1, 'Deluxe': 2, 'Super Deluxe': 3, 'King': 4}
marital_status_map = {'Single': 0, 'Married': 1, 'Divorced': 2, 'Unmarried': 3}
designation_map = {'Executive': 0, 'Manager': 1, 'Senior Manager': 2, 'AVP': 3, 'VP': 4}

# Convert categorical inputs to match model training
input_data = pd.DataFrame([{
    'Age': Age,
    'TypeofContact': type_of_contact_map[TypeofContact],
    'CityTier': CityTier,
    'DurationOfPitch': DurationOfPitch,
    'Occupation': occupation_map[Occupation],
    'Gender': gender_map[Gender],
    'NumberOfPersonVisiting': NumberOfPersonVisiting,
    'NumberOfFollowups': NumberOfFollowups,
    'ProductPitched': product_pitched_map[ProductPitched],
    'PreferredPropertyStar': PreferredPropertyStar,
    'MaritalStatus': marital_status_map[MaritalStatus],
    'NumberOfTrips': NumberOfTrips,
    'Passport': 1 if Passport == "Yes" else 0,
    'PitchSatisfactionScore': PitchSatisfactionScore,
    'OwnCar': 1 if OwnCar == "Yes" else 0,
    'NumberOfChildrenVisiting': NumberOfChildrenVisiting,
    'Designation': designation_map[Designation],
    'MonthlyIncome': MonthlyIncome
}])

# Predict button
if st.button("Predict"):
    prediction = model.predict(input_data)[0]
    result = "purchase the package" if prediction == 1 else "not purchase the package"
    st.write(f"Based on the information provided, the customer is likely to {result}.")

## Dependency Handling

In [ ]:
%%writefile tourism_project/deployment/requirements.txt

pandas==2.2.2
huggingface_hub==0.32.6
streamlit==1.43.2
joblib==1.5.1
scikit-learn==1.6.0
xgboost==2.1.4
mlflow==3.0.1

# Hosting

In [ ]:
os.makedirs("tourism_project/hosting", exist_ok=True)

In [ ]:
%%writefile tourism_project/hosting/hosting.py

from huggingface_hub import HfApi
import os

api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="tourism_project/deployment",     # the local folder containing your files
    repo_id="<-----Hugging Face User ID ----->/tourism-package-prediction",          # the target repo
    repo_type="space",                      # dataset, model, or space
    path_in_repo="",                          # optional: subfolder path inside the repo
)

# Create and Automate MLOps Pipeline with GitHub Action Workflows using CI/CD
## Actions Workflow YAML File
* A YAML file is a simple, human-readable file used to store configuration settings.

* YAML stands for Yet Another Markup Language or YAML Ain't Markup Language (a recursive acronym).

* It uses indentation (spaces) to show structure, like folders inside folders.

* Each line contains a key and a value, making it easy to organize data.

* YAML is often used in automation tools, cloud setups, and app settings.

Here's the YAML file we'd need for our use case.
```

name: Tourism MLOps pipeline



on:

  push:

    branches:

      - main  # Automatically triggers on push to the main branch

jobs:

  register-dataset:

    runs-on: ubuntu-latest

    steps:

      - uses: actions/checkout@v3

      - name: Install Dependencies

        run: pip install -r tourism_project/requirements.txt

      - name: Upload Dataset to Hugging Face Hub

        env:

          HF_TOKEN: ${{ secrets.HF_TOKEN }}

        run: python tourism_project/model_building/data_register.py



  data-prep:

    needs: register-dataset

    runs-on: ubuntu-latest

    steps:

      - uses: actions/checkout@v3

      - name: Install Dependencies

        run: pip install -r tourism_project/requirements.txt

      - name: Run Data Preparation

        env:

          HF_TOKEN: ${{ secrets.HF_TOKEN }}

        run: python tourism_project/model_building/prep.py





  model-traning:

    needs: data-prep

    runs-on: ubuntu-latest

    steps:

      - uses: actions/checkout@v3

      - name: Install Dependencies

        run: pip install -r tourism_project/requirements.txt

      - name: Start MLflow Server

        run: |

          nohup mlflow ui --host 0.0.0.0 --port 5000 &  # Run MLflow UI in the background

          sleep 5  # Wait for a moment to let the server start

      - name: Model Building

        env:

          HF_TOKEN: ${{ secrets.HF_TOKEN }}

        run: python tourism_project/model_building/train.py



  deploy-hosting:

    runs-on: ubuntu-latest

    needs: [model-traning,data-prep,register-dataset]

    steps:

      - uses: actions/checkout@v3

      - name: Install Dependencies

        run: pip install -r tourism_project/requirements.txt

      - name: Push files to Frontend Hugging Face Space

        env:

          HF_TOKEN: ${{ secrets.HF_TOKEN }}

        run: python tourism_project/hosting/hosting.py



```
**Note:** To use this YAML file for our use case, we need to



1. Go to the GitHub repository for the project

2. Create a folder named ***.github/workflows/***

3. In the above folder, create a file named ***pipeline.yml***

4. Copy and paste the above content for the YAML file into the ***pipeline.yml*** file

## Requirements file for the Github Actions Workflow

In [ ]:
%%writefile tourism_project/requirements.txt

huggingface_hub==0.32.6
datasets==3.6.0
pandas==2.2.2
scikit-learn==1.6.0
xgboost==2.1.4
mlflow==3.0.1

## Github Authentication and Push Files
* Before moving forward, we need to generate a secret token to push files directly from Colab to the GitHub repository.

* Please follow the below instructions to create the GitHub token:

    - Open your GitHub profile.

    - Click on ***Settings***.

    - Go to ***Developer Settings***.

    - Expand the ***Personal access tokens*** section and select ***Tokens (classic)***.

    - Click ***Generate new token***, then choose ***Generate new token (classic)***.

    - Add a note and select all required scopes.

    - Click ***Generate token***.

    - Copy the generated token and store it safely in a notepad.

In [ ]:
# Install Git
!apt-get install git

# Set your Git identity (replace with your details)
!git config --global user.email "<-----GitHub email address----->"
!git config --global user.name "<-----GitHub username----->"

# Clone your GitHub repository
!git clone https://github.com/<-----GitHub username----->/<-----GitHub repo name----->.git

# Move your folder to the repository directory
!mv /content/tourism_project/ /content/<-----GitHub repo name----->

In [ ]:
# Change directory to the cloned repository
%cd <-----GitHub repo name----->/

# Add the new folder to Git
!git add .

# Commit the changes
!git commit -m "first commit"

# Push to GitHub (you'll need your GitHub credentials; use a personal access token if 2FA enabled)
!git push https://<-----GitHub username----->:<-----GitHub token ----->@github.com/<-----GitHub username----->/<-----GitHub repo name----->.git

# Output Evaluation
- GitHub (link to repository, screenshot of folder structure and executed workflow)

- Streamlit on Hugging Face (link to HF space, screenshot of Streamlit app)

<font size=6 color="navyblue">Power Ahead!</font>

___